In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import sys
sys.path.insert(0,'/Users/marcelvnguemaha/repos/cmsdatajam/codes')
import warnings
warnings.filterwarnings('ignore')
#import app

- [guide to FIPS codes](https://plotly.com/python/county-choropleth/)
- [State and County FIPS data](https://towardsdatascience.com/the-ultimate-state-county-fips-tool-1e4c54dc9dff)
- [State and County FIPS GIT](https://github.com/ChuckConnell/articles/blob/master/fips2county.tsv) - fips2county.tsv comes from here
- [CDC Stats on CKD in adults](https://www.cdc.gov/kidneydisease/publications-resources/ckd-national-facts.html) - national prevalence about 15%
- [world atlas cool project](https://github.com/danny-baker/atlas)

In [4]:


dtypes = {
    'StateFIPS':str, 
    'CountyFIPS_3':str, 
    'CountyName':str, 
    'StateName':str, 
    'CountyFIPS':str,
    'StateAbbr':str, 
    'STATE_COUNTY':str
}

ckd = pd.read_csv('https://raw.githubusercontent.com/nmmarcelnv/cmsdatajam/main/data/Prevalence_of_CKD_by_US_State_and_County_by_County_2019.csv')
fips = pd.read_csv('https://raw.githubusercontent.com/ChuckConnell/articles/master/fips2county.tsv', sep='\t', usecols=['CountyName','StateName','CountyFIPS','StateAbbr']
                  ,dtype=dtypes)

fips.columns=['county','state','fips', 'state3']
fips = fips[['state','county','fips', 'state3']]
fips['county'] = fips['county'].str.upper()
fips['state'] = fips['state'].str.upper()


ckd.columns=['cases','county','state']
ckd['county'] = ckd['county'].str.upper()
ckd['state'] = ckd['state'].str.upper()
ckd = ckd[['state','county','cases']]


print(len(fips['county'].unique()))
print(len(ckd['county'].unique()))

1878
1833


In [5]:
df = pd.merge(ckd, fips,on=['state','county'])

In [6]:
print(df.shape, ckd.shape)

(2996, 5) (3123, 3)


In [7]:
df.head()

,state,county,cases,fips,state3
0,ALABAMA,AUTAUGA,22.35,01001,AL
1,ALABAMA,BALDWIN,21.21,01003,AL
2,ALABAMA,BARBOUR,23.53,01005,AL
3,ALABAMA,BIBB,37.21,01007,AL
4,ALABAMA,BLOUNT,31.03,01009,AL


In [9]:
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [10]:
#https://plotly.com/python/choropleth-maps/

In [11]:
#df.loc[df.cases<80, 'cases'] = df.loc[df.cases<80, 'cases']*0.1

In [12]:

fig = px.choropleth(
    df, 
    geojson="https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json",
    locations="fips", 
    color='cases',
    scope='usa',
    color_continuous_scale='YlOrRd',
    range_color=(20, 40),
    hover_data = {'state':True,'county':True},
    labels={'cases':'CKD Prevalence (%)'}
)

fig.layout.template = None
fig.show()

In [13]:
import json
import pandas as pd
import urllib.request

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
with urllib.request.urlopen(f'{url}/us_counties_20m_topo.json') as f:
    counties = json.load(f)

In [14]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

counties["features"][0]

{'type': 'Feature',
 'properties': {'GEO_ID': '0500000US01001',
  'STATE': '01',
  'COUNTY': '001',
  'NAME': 'Autauga',
  'LSAD': 'County',
  'CENSUSAREA': 594.436},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-86.496774, 32.344437],
    [-86.717897, 32.402814],
    [-86.814912, 32.340803],
    [-86.890581, 32.502974],
    [-86.917595, 32.664169],
    [-86.71339, 32.661732],
    [-86.714219, 32.705694],
    [-86.413116, 32.707386],
    [-86.411172, 32.409937],
    [-86.496774, 32.344437]]]},
 'id': '01001'}

In [15]:
df.to_csv('../data/ckd_by_county.csv', index=False)